## Замечания для клевера:
Актуально:
* Можно улучшить обработку текстовых данных (можно попробовать в сыром виде запихать в кэтбуст)
* Если лемматизация будет работать долго (есть такая вероятность) - можно поменять на стеммер

Старое:
* Вариант с тем, что в одной колонке название характеристики, в другой значение будет работать плохо (вот пример), т.к. модели без разницы на порядок следования колонок
* Правильно ли я понимаю, что все колонки кроме ХК 1 и целевой имеют тип данных String?
* **Как предсказывать строки с пустыми значениями во всех колонках ХК? (может их сразу откидывать)?**
* Названия первой колонки должны быть всегда одинаковые
* !!!Важно!!! Будем заменять числовые факторы на категориальные, если в них маленькое количество уникальных значений или одно значение встречается очень часто
* Были ошибки в названиях колонок: 'ХК_ка т_01'

**Проблемы, решение которых нужно будет автоматизировать:**
* Несбалансированность классов
* Пропуски в данных
* Автоматическое кодирование текстовых столбцов

In [1]:
import pandas as pd
import numpy as np
from loguru import logger
import re
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer

#nltk.download('punkt')
#nltk.download('stopwords')

#### Глобальные переменные

In [2]:
# Максимальное количество уникальных значений для категориального фактора, при котором он может обрабатываться методом one-hot encoding (добавится максимум столько столбцов)
OneHotEncodingLimit = 30

### Работа с данными

In [3]:
df = pd.read_excel('data/paper_classificator_data.xlsx')

In [4]:
df = df[df['ID класса (ТАРГЕТ)'].notna()]

In [5]:
df.head(1)

,ID класса (ТАРГЕТ),Наименование терминального класса,Код родительского класса,Наименование родительского класса,Историческое наименование,ХК_Кат_01,Значение ХК_Кат_01,ХК_Кат_02,Значение ХК_Кат_02,ХК_Кат_03,Значение ХК_Кат_03,ХК_Стр_01,Значение ХК_Стр_01,ХК_Числ_01,Значение ХК_Числ_01,ХК_Числ_02,Значение ХК_Числ_02,ХК_Числ_03,Значение ХК_Числ_03
0,12326143.0,"Бумага для офисной техники листовая цветная, А4",01.15.01,БУМАГА,"Бумага д/принтера цветная IQ Color, А4, 80г/м2...",Производитель,IQ Color,Формат,A4,Цвет,розовый,NaN,NaN,Листов в пачке,100.0,"Плотность, г/м2",80.0,NaN,NaN


In [6]:
target = df['ID класса (ТАРГЕТ)']

# Удалим все лишние текстовые столбцы кроме "Исторического наименования"
trainset_columns = []
for column in df.columns:
    if (column == 'Историческое наименование') or (re.fullmatch(r'ХК_.*', column)!=None) or (re.fullmatch(r'Значение.*', column)!=None):
        trainset_columns.append(column)

factors_df = df[trainset_columns]

#### Работаем с типами данных столбцов

In [7]:
def format_column_types(columns: list):
    '''
    Обрабатывает названия колонок из массива columns.
    Возвращает словарь с парами: название колонки - ее тип данных  
    '''
    feature_types_dict = {}
    for column in columns:
        type_pattern = r'ХК_([^_]+)_.*'
        if column[0:2] == 'ХК':
            feature_types_dict[column] = 'Кат'
        elif column[0:8]=='Значение':
            column_type = re.findall(type_pattern, column)[0]
            feature_types_dict[column] = column_type
        else:
            feature_types_dict[column] = 'Стр'
    return feature_types_dict

feature_types_dict = format_column_types(factors_df.columns)

In [8]:
def check_number_to_categorical(column: str, factor: pd.Series):
    logger.info(f'Начинаем проверку численного фактора {column}\n')
    logger.debug(f'Размер фактора:{factor.size}')
    logger.debug(f'Количество уникальных значений: {factor.drop_duplicates().size}')
    logger.debug(f'Процент заполненности фактора: {factor[factor.notnull()].size / factor.size * 100}%')
    popular_value = pd.DataFrame(factor.value_counts().sort_values(ascending=False).head(1)/factor[factor.notnull()].size*100)
    popular_value.columns = ['Частота']
    logger.debug(f'Cамое частое значение фактора: \n{popular_value}')
    
    if float(popular_value.iloc[0])>=50:
        logger.info(f'Переводим числовой фактор {column} в категориальный')
        return True
    
    
for feature in feature_types_dict.keys():
    if feature_types_dict.get(feature) == 'Стр':
        logger.debug(f'Строковый фактор: {feature}')
        #todo ДОДЕЛАТЬ пока ничего не делаем, чтобы не потерять пропущенные значения при преобразовании в строковый формат
        factors_df[feature] = factors_df[feature].astype(object)
    elif feature_types_dict.get(feature) == 'Булево':
        factors_df[feature] = factors_df[feature].astype(bool)
    elif feature_types_dict.get(feature) == 'Числ':
        if check_number_to_categorical(feature, factors_df[feature]):
            feature_types_dict[feature] = 'Кат'
            factors_df[feature] = factors_df[feature].astype(object)
        else:   
            factors_df[feature] = factors_df[feature].astype(float)
    elif feature_types_dict.get(feature) == 'Кат':
        logger.debug(f'Категориальный фактор: {feature}')
        #todo ДОДЕЛАТЬ преобразование категориальных колонок (пока не делаем, т.к. возможно будет catboost)


2023-10-09 02:07:42.405 | DEBUG    | __main__:<module>:17 - Строковый фактор: Историческое наименование
C:\Users\User\AppData\Local\Temp\ipykernel_18560\133429103.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factors_df[feature] = factors_df[feature].astype(object)
2023-10-09 02:07:42.407 | DEBUG    | __main__:<module>:29 - Категориальный фактор: ХК_Кат_01
2023-10-09 02:07:42.407 | DEBUG    | __main__:<module>:29 - Категориальный фактор: Значение ХК_Кат_01
2023-10-09 02:07:42.408 | DEBUG    | __main__:<module>:29 - Категориальный фактор: ХК_Кат_02
2023-10-09 02:07:42.408 | DEBUG    | __main__:<module>:29 - Категориальный фактор: Значение ХК_Кат_02
2023-10-09 02:07:42.409 | DEBUG    | __main__:<module>:29 - Категориальный фактор: ХК_Кат_03


In [9]:
#!pip install pymystem3

#### Заполняем пропуски в данных
В зависимости от типа данных колонки заполняем пропуски по-разному:
*   Стр -  т.к. переводим строки в числа, то пропущенные значение пусть будут = 0
*   Числ - #todo По умолчанию = 0. Если присутствует значение, количество которого в заполненных строках >=50% => то фактор станет категориальным, а не численным. 
*   Булево - #todo будем считать, что у нас всегда такие столбцы отвечают на вопрос: "Есть что-то? - Да/Нет". Если нет ответа => Нет
*   Кат - 'Emptyclass'

In [10]:
factors_df.isna().sum()

Историческое наименование      0
ХК_Кат_01                    151
Значение ХК_Кат_01           151
ХК_Кат_02                     40
Значение ХК_Кат_02            40
ХК_Кат_03                    130
Значение ХК_Кат_03           130
ХК_Стр_01                    191
Значение ХК_Стр_01           191
ХК_Числ_01                   151
Значение ХК_Числ_01          151
ХК_Числ_02                   137
Значение ХК_Числ_02          137
ХК_Числ_03                   147
Значение ХК_Числ_03          147
dtype: int64

In [11]:
not_empty_factors_df = factors_df.copy()
for column in not_empty_factors_df.columns:
    if feature_types_dict.get(column) == 'Кат':
        not_empty_factors_df.loc[not_empty_factors_df[column].isna(), column] = f'EmptyValue_{column}'
    elif feature_types_dict.get(column) == 'Числ':
        not_empty_factors_df.loc[not_empty_factors_df[column].isna(), column] = 0
    elif feature_types_dict.get(column) == 'Стр':
        not_empty_factors_df.loc[not_empty_factors_df[column].isna(), column] = ''
    elif feature_types_dict.get(column) == 'Булево':
        not_empty_factors_df.loc[not_empty_factors_df[column].isna(), column] = 0


In [12]:
not_empty_factors_df.isna().sum().sort_values()

Историческое наименование    0
ХК_Кат_01                    0
Значение ХК_Кат_01           0
ХК_Кат_02                    0
Значение ХК_Кат_02           0
ХК_Кат_03                    0
Значение ХК_Кат_03           0
ХК_Стр_01                    0
Значение ХК_Стр_01           0
ХК_Числ_01                   0
Значение ХК_Числ_01          0
ХК_Числ_02                   0
Значение ХК_Числ_02          0
ХК_Числ_03                   0
Значение ХК_Числ_03          0
dtype: int64

#### Кодируем строковые переменные
Возможные варианты:
* bag_of_words - пока остановимся на нем
* tf_idf 

In [13]:
russian_stopwords = stopwords.words("russian")
mystem = Mystem()

def text_preprocessing(text):
    tokens = mystem.lemmatize(text)
    tokens = [token for token in tokens if token not in russian_stopwords and token != " "  and token.strip() not in punctuation]
    
    return tokens

def text_feature_preprocessing(text_feature):
    '''
    Функция преобразования текстовых факторов
    - Переводим в нижний регистр
    - Удаляем знаки препинания
    - Удаляем стоп слова
    - Проводим лемматизацию
    '''
    processed_feature = []
    text_feature = text_feature.replace(r'[^\w\s]',' ', regex=True).replace(r'\s+',' ', regex=True).str.lower()
    processed_text_feature = text_feature.apply(text_preprocessing)
    return processed_text_feature

def handle_text_feature(text_feature: pd.Series):
    '''
    Функция обработки строкового фактора:
    - Проводим препроцессинг
    - Формируем "Мешок строк" (bag of words)
    '''
    processed_text_feature = text_feature_preprocessing(text_feature)

    vectorizer = CountVectorizer()
    vectorizer.fit(processed_text_feature)
    vectorized_text_feature = pd.DataFrame(vectorizer.transform(processed_text_feature).toarray())

    # Удалим неинформативные столбцы
    informative_word_columns = vectorized_text_feature.sum()[
            (vectorized_text_feature.sum()>=vectorized_text_feature.shape[1]*0.01) &
            (vectorized_text_feature.sum()!=vectorized_text_feature.shape[1])
        ].index 
    handled_text_feature = vectorized_text_feature[informative_word_columns]
    handled_text_feature.columns = pd.Series(vectorizer.get_feature_names_out())[informative_word_columns]
    return handled_text_feature

In [14]:
from sklearn.preprocessing import LabelEncoder

def handle_cat_feature(cat_feature: pd.Series):
    cat_feature = cat_feature.astype(str)
    unique_values_count = cat_feature.drop_duplicates().size
    if unique_values_count <= OneHotEncodingLimit:
        #OneHotEncoding
        cat_feature_encoded = pd.get_dummies(cat_feature)
    else:
        #LabelEncoding - чтобы сильно не увеличивать количество факторов
        le = LabelEncoder()
        cat_feature_encoded = pd.DataFrame(le.fit_transform(cat_feature))
    return cat_feature_encoded

In [15]:
trainset = pd.DataFrame()

for feature in feature_types_dict:
    if feature_types_dict.get(feature) == 'Стр':
        handled_feature = handle_text_feature(not_empty_factors_df[feature])
    elif feature_types_dict.get(feature) == 'Кат':
        handled_feature = handle_cat_feature(not_empty_factors_df[feature])
    else:
        handled_feature = pd.DataFrame(not_empty_factors_df[feature])
    handled_feature.columns = [col+'_'+feature for col in handled_feature.columns]
        
    trainset = pd.concat([trainset,handled_feature],axis=1)

In [16]:
trainset.head()

,10_Историческое наименование,100_Историческое наименование,100л_Историческое наименование,10л_Историческое наименование,10цв_Историческое наименование,12_Историческое наименование,146_Историческое наименование,170cie_Историческое наименование,1school_Историческое наименование,20_Историческое наименование,...,170.0_Значение ХК_Числ_02,180.0_Значение ХК_Числ_02,250.0_Значение ХК_Числ_02,280.0_Значение ХК_Числ_02,65.0_Значение ХК_Числ_02,80.0_Значение ХК_Числ_02,EmptyValue_Значение ХК_Числ_02_Значение ХК_Числ_02,EmptyValue_ХК_Числ_03_ХК_Числ_03,"Белизна по CIE, %_ХК_Числ_03",Значение ХК_Числ_03_Значение ХК_Числ_03
0,0,0,1,0,0,0,0,0,0,0,...,False,False,False,False,False,True,False,True,False,0.0
1,0,0,1,0,0,0,0,0,0,0,...,False,False,False,False,False,True,False,True,False,0.0
2,0,0,1,0,0,0,0,0,0,0,...,False,False,False,False,False,True,False,True,False,0.0
3,0,0,1,0,0,0,0,0,0,0,...,False,False,False,False,False,True,False,True,False,0.0
4,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,True,False,True,False,0.0


#### Сэмплируем классы, в которых всего 1 экземпляр и делим на обучающую и тестовую выборки

In [24]:
#!pip install imblearn

In [42]:
from sklearn.utils import resample 

trainset = pd.concat([trainset, target], axis = 1)

for label in target.drop_duplicates():
    if target[target == label].size == 1:
        minor_target_df = trainset[trainset['ID класса (ТАРГЕТ)']==label]
        print(minor_target_df)


ValueError: cannot reindex on an axis with duplicate labels

In [27]:
target.shape

(211,)

In [29]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(k_neighbors=2, random_state=42)
trainset_resampled, target_resampled = smote.fit_resample(trainset, target)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 1, n_neighbors = 3

In [17]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_labels, test_labels = train_test_split(
    trainset, 
    target, 
    test_size=0.3, 
    random_state=42,
    #stratify=target    
) 

#### Обучим модель CatBoostClassifier на подготовленных данных

In [18]:
from catboost import CatBoostClassifier

In [21]:
target.value_counts()

ID класса (ТАРГЕТ)
12308627.0    56
3288708.0     28
4177853.0     16
12326143.0    11
4283045.0      9
12363346.0     7
4186617.0      7
3293136.0      7
12363587.0     6
8956452.0      6
12344482.0     6
12363576.0     5
12363146.0     5
12345290.0     5
12363315.0     4
12345190.0     4
12344579.0     4
12363481.0     4
12362113.0     4
12363185.0     4
12308694.0     4
12363366.0     4
3296939.0      2
3291817.0      1
3596636.0      1
3293134.0      1
Name: count, dtype: int64

In [19]:
model = CatBoostClassifier(loss_function='MultiClass', verbose=True)

grid = {'learning_rate': [0.03, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
        'depth': [4, 6, 7, 8, 9, 10, 11, 12, 13, 14],
        #'l2_leaf_reg': [1, 3, 5, 7, 9]
        'iterations':[2,3,4,5,6,7,8,9,10,11,12,13]
        }

grid_search_result = model.grid_search(
        grid,
        X=train_data,
        y=train_labels,
)

0:	learn: 3.2165563	test: 3.2292190	best: 3.2292190 (0)	total: 64.8ms	remaining: 64.8ms
1:	learn: 3.1682473	test: 3.1722732	best: 3.1722732 (1)	total: 70.6ms	remaining: 0us

bestTest = 3.172273208
bestIteration = 1

0:	loss: 3.1722732	best: 3.1722732 (0)	total: 108ms	remaining: 1m 30s
0:	learn: 3.1891115	test: 3.2101587	best: 3.2101587 (0)	total: 15.5ms	remaining: 15.5ms
1:	learn: 3.1102434	test: 3.1171741	best: 3.1171741 (1)	total: 21.2ms	remaining: 0us

bestTest = 3.117174132
bestIteration = 1

1:	loss: 3.1171741	best: 3.1171741 (1)	total: 131ms	remaining: 55s
0:	learn: 3.1214689	test: 3.1632433	best: 3.1632433 (0)	total: 5.98ms	remaining: 5.98ms
1:	learn: 2.9725075	test: 2.9874406	best: 2.9874406 (1)	total: 12.2ms	remaining: 0us

bestTest = 2.987440587
bestIteration = 1

2:	loss: 2.9874406	best: 2.9874406 (2)	total: 145ms	remaining: 40.6s
0:	learn: 3.0554061	test: 3.1175064	best: 3.1175064 (0)	total: 13.5ms	remaining: 13.5ms
1:	learn: 2.8461309	test: 2.8700521	best: 2.8700521 (1)	to

CatBoostError: C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/private/libs/algo/data.cpp:196: Dataset test #0 contains class label "3.29313e+06" that is not present in the learn dataset

In [44]:
model.fit(train_data, train_labels)
preds_labels = model.predict(test_data)

Learning rate set to 0.071756
0:	learn: 3.1336563	total: 23ms	remaining: 23s
1:	learn: 3.0226106	total: 42.4ms	remaining: 21.2s
2:	learn: 2.9360590	total: 64.9ms	remaining: 21.6s
3:	learn: 2.8661307	total: 85.5ms	remaining: 21.3s
4:	learn: 2.7492422	total: 107ms	remaining: 21.2s
5:	learn: 2.6685978	total: 126ms	remaining: 20.9s
6:	learn: 2.5917673	total: 145ms	remaining: 20.6s
7:	learn: 2.5321914	total: 165ms	remaining: 20.4s
8:	learn: 2.4641664	total: 185ms	remaining: 20.4s
9:	learn: 2.4217673	total: 205ms	remaining: 20.3s
10:	learn: 2.3311116	total: 230ms	remaining: 20.7s
11:	learn: 2.2815793	total: 258ms	remaining: 21.2s
12:	learn: 2.2431107	total: 278ms	remaining: 21.1s
13:	learn: 2.2013459	total: 300ms	remaining: 21.1s
14:	learn: 2.1528092	total: 320ms	remaining: 21s
15:	learn: 2.1085195	total: 342ms	remaining: 21s
16:	learn: 2.0588412	total: 364ms	remaining: 21s
17:	learn: 2.0153665	total: 387ms	remaining: 21.1s
18:	learn: 1.9907399	total: 408ms	remaining: 21.1s
19:	learn: 1.9508

In [45]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(test_labels, preds_labels)
report = classification_report(test_labels, preds_labels)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.84375
Classification Report:
               precision    recall  f1-score   support

   3288708.0       0.80      0.89      0.84         9
   3293136.0       1.00      0.50      0.67         2
   4177853.0       0.75      0.60      0.67         5
   4186617.0       1.00      0.67      0.80         3
   4283045.0       0.25      1.00      0.40         1
   8956452.0       0.00      0.00      0.00         2
  12308627.0       1.00      1.00      1.00        14
  12308694.0       1.00      1.00      1.00         3
  12326143.0       1.00      1.00      1.00         2
  12344482.0       1.00      0.75      0.86         4
  12344579.0       0.75      1.00      0.86         3
  12345190.0       1.00      1.00      1.00         1
  12345290.0       1.00      1.00      1.00         2
  12362113.0       1.00      1.00      1.00         1
  12363146.0       1.00      1.00      1.00         1
  12363185.0       1.00      1.00      1.00         2
  12363315.0       1.00      1.00      

#### Удалим бесполезные факторы которые состоят из 1 уникального значения

In [ ]:
#Делаем это после кодирования категориалььных переменных
#for column in factors_df.columns:
#    unique_values_count = factors_df[column].drop_duplicates().size
#    if unique_values_count == 1:
#        new_factors_df = factors_df.drop(column, axis = 1)

In [ ]:
#new_factors_df.info()

In [ ]:
#new_factors_df['Наименование терминального класса'].value_counts()

* Удалить колонки для неполного соответствия: если есть [1,2,3,4] - удалить одну, чтобы не было зависимости
* Удалить строки без характеристик
* Отбор на основе важности признаков